In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

In [2]:
URL = "https://www.amazon.com/s?k=Monitor&crid=285CHRYI51VOQ&sprefix=monitor%2Caps%2C110&ref=nb_sb_noss_1"

In [3]:
HEADERS = ({'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36','Accept-Language' : 'en-US en:q=0.5'})

In [4]:
webpage = requests.get(URL,headers = HEADERS)

In [5]:
soup = BeautifulSoup(webpage.content,"html.parser")

In [6]:
links = soup.find_all("a",attrs={"class" : "a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"})

In [7]:
def getTitle(product_soup):
    try :
        title = product_soup.find('span',attrs = {"class":"a-size-large product-title-word-break"})
        return title.text.strip()
    except:
        return "" 

def getPrice(product_soup):
    try:
        price = product_soup.find('span',attrs = {"class":"a-offscreen"})
        return price.text
    except:
        return ""

def getRatings(product_soup):
    try:
        ratings = product_soup.find('span',attrs = {"class":"a-icon-alt"})
        return ratings.text[:3]
    except:
        return ""

def getAvailability(product_soup):
    try:
        Availability = product_soup.find('span',attrs = {"class":"a-size-medium a-color-success"})
        return Availability.text.strip()
    except:
        return ""

def getNumOfReviews(product_soup):
    try:
        Reviews = product_soup.find('span',attrs={"id":"acrCustomerReviewText"})
        return Reviews.text[:2]
    except:
        return ""


In [8]:
#Monitors = pd.DataFrame(columns = ["Title",'Price',"Ratings","Reviews","Availability","Link"])
dict = {'Title':[],"Price":[],'Ratings':[],"Reviews":[],
           'Availability':[],"Link":[]}
for link in links:
    product_link = "https://amazon.com" + link.get('href')
    product_webpage = requests.get(product_link,headers=HEADERS)
    product_soup = BeautifulSoup(product_webpage.content,"html.parser")
    product_title = getTitle(product_soup)
    product_price = getPrice(product_soup)
    product_ratings = getRatings(product_soup)
    product_availability = getAvailability(product_soup)
    product_num_of_reviews = getNumOfReviews(product_soup)
    dict['Title'].append(product_title)
    dict["Price"].append(product_price)
    dict['Ratings'].append(product_ratings)
    dict['Reviews'].append(product_num_of_reviews)
    dict['Availability'].append(product_availability)
    dict['Link'].append(product_link)



In [35]:
Monitors = pd.DataFrame.from_dict(dict)
Monitors.head()

,Title,Price,Ratings,Reviews,Availability,Link
0,Amazon Basics 27 Inch Monitor Powered with AOC...,$164.99,4.5,98,In Stock,https://amazon.com/gp/slredirect/picassoRedire...
1,KOORUI 24 Inch Computer Monitor Full HD 1920 x...,$95.99,4.3,"2,",,https://amazon.com/gp/slredirect/picassoRedire...
2,"Sceptre 20"" 1600 x 900 75Hz LED Monitor 2x HDM...",$67.97,4.5,25,In Stock,https://amazon.com/Sceptre-Monitor-1600x900-Sp...
3,"Sceptre 24"" Professional Thin 75Hz 1080p LED M...",$99.97,4.6,25,In Stock,https://amazon.com/Sceptre-E248W-19203R-Monito...
4,Acer 21.5 Inch Full HD (1920 x 1080) IPS Ultra...,$88.68,4.6,41,In Stock,https://amazon.com/Acer-SB220Q-Ultra-Thin-Fram...


In [36]:
Monitors.to_csv("Amazon_Monitors.csv",header=True,index=False)